In [1]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 모델 로드
model = BertForSequenceClassification.from_pretrained('./saved_model')
tokenizer = BertTokenizer.from_pretrained('./saved_model')

# 모델 학습
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

# 데이터셋 정의
class HateSpeechDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        label = self.labels[idx]
        
        # 1~5 label을 True, 0 label을 False로 변환
        if label in [1, 2, 3, 4, 5]:
            label = 1
        else:
            label = 0
        
        return {
            'input_ids': inputs['input_ids'].flatten().to(device),
            'attention_mask': inputs['attention_mask'].flatten().to(device),
            'labels': torch.tensor(label, dtype=torch.long).to(device)
        }
    

# 데이터 로드
df = pd.read_csv('data/output.csv')

# 데이터를 1000개로 제한
X_train, X_test, y_train, y_test = train_test_split(df['text'].sample(1000), df['types'].sample(1000), test_size=0.2, random_state=42)

train_dataset = HateSpeechDataset(X_train.tolist(), y_train.tolist(), tokenizer)
test_dataset = HateSpeechDataset(X_test.tolist(), y_test.tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


# 평가
model.eval()
predictions, true_labels = [], []
num = 0
for batch in test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    batch_predictions = torch.argmax(logits, dim=1).tolist()
    batch_true_labels = batch['labels'].tolist()
    
    for i, (text, pred, true_label) in enumerate(zip(X_test, batch_predictions, batch_true_labels)):
        if pred == true_label:
            num += 1
        print(f"Text: {text}")
        print(f"Predicted Label: {pred}")
        print(f"True Label: {true_label}")
        print()

accuracy = num / len(X_test)
print(f"Accuracy: {accuracy:.2f}")


Text: #@#이 일단 유명해져라, 그러면 똥을 싸도 박수를 칠 것이다라고 말한 대표적인 예
Predicted Label: 5
True Label: 1

Text: 엄마랑 야스중이라서 못간다고해
Predicted Label: 1
True Label: 1

Text: 니들처럼 드러운 짓은 안한다
Predicted Label: 5
True Label: 1

Text: 한남충한테 오빠는 좀... 과한 용어가 아닌지요
Predicted Label: 1
True Label: 1

Text: 여자를 못 사귄 게 다 일회용이라 생각해서 그런 거지
Predicted Label: 5
True Label: 1

Text: 나 지금 버스에 앉아있는데 늙딱룐이 앞에서 쳐다보는 거 개킹받음 하
Predicted Label: 4
True Label: 1

Text: 그러니까 난민 받아주면 우리가 다 듸진다
Predicted Label: 4
True Label: 1

Text: 난 그러면 허영심 가득한 저능아들 같아서 별로던데
Predicted Label: 5
True Label: 1

Text: 중국 사람들 잘 안씻어서 냄새나고 그러지 않아?
Predicted Label: 1
True Label: 1

Text: 아 레발 그런 련들은 사고 한번 크게 나야지 정신 차리는 건가 리발
Predicted Label: 5
True Label: 1

Text: 너의 그 가오부리면서 헛소리 받아 준다고 나는 정말 토쏠렸어
Predicted Label: 5
True Label: 1

Text: 쟤 뭐만하면 화내는 거 거의 분조장 아니냐?
Predicted Label: 4
True Label: 1

Text: 응 그 남자들 너한테는 안 와~
Predicted Label: 5
True Label: 1

Text: 발뺌 하나는 기가 막히군 혀를 뽑아버려야 돼.
Predicted Label: 3
True Label: 1

Text: 난 대머리들 변태 노인네 같아서 싫단 말이야
Predicted Label